In [ ]:
import os.path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from hit_prediction_code.analytics import get_results_as_dataframe
from hit_prediction_code import analytics

results = get_results_as_dataframe(
    project_name='hit-prediction-ismir2020',
    table_name='hit_prediction',
    filter_git_dirty=False,
    date_filter='> \'2021-08-19 13:20:00\'',
    columns=['id', 'date', 'sourcefile', 'outcome'],
#     filters=['sourcefile LIKE \'plans/hspd/regression_%%.py\''],
    filters=['sourcefile LIKE \'plans/hspd/regression_%%_mel%%.py\''],
#     filters=['sourcefile LIKE \'plans/ordinal%%\''],
)

analytics.add_approach_to_df(results)
analytics.add_cv_epoch_evaluator_outcome_to_df(results)

results.sort_values(by='sourcefile', inplace=True)
display(len(results))

In [ ]:
best_results = []
for _, row in results.iterrows():
    metrics = list(filter(lambda i: i not in ['confusion_matrix'], row['mean'].index))
    row = pd.DataFrame(pd.concat([row, row['mean'].loc[metrics].max(axis=1).abs()], axis=0)).transpose()

    best_results.append(row)
best_results = pd.concat(best_results)

best_results['mae'] = best_results['neg_mean_absolute_error']
best_results['meae'] = best_results['neg_median_absolute_error']
best_results['rmse'] = best_results['neg_mean_squared_error'].pow(1./2)
best_results = best_results.sort_values(by=['sourcefile'])

In [ ]:
metric_avg = analytics.aggregate_splits_per_epoch(results['outcome'], np.average)

# display(analytics.aggregate_epochs(metric_avg, np.max).abs())

In [ ]:
best_results['approach'] = best_results['approach'].apply(lambda v: v.replace('mean_std', 'ms'))
# display(best_results[['date', 'approach', 'mae', 'rmse', 'pearsonr', 'spearmanr', 'kendalltau']])
display(best_results[['approach', 'spearmanr', 'kendalltau']])

# display(best_results[['approach', 'mae']].plot.bar(x='approach', y='mae', title='Wide and Deep MAE', figsize=(12, 6)))
# display(best_results[['approach', 'rmse']].plot.bar(x='approach', y='rmse', title='Wide and Deep RMSE', figsize=(12, 6)))

# display(best_results[['approach', 'pearsonr']].plot.bar(x='approach', y='pearsonr', title='Wide and Deep Pearson Correlation', figsize=(12, 6)))
display(best_results[['approach', 'spearmanr']].plot.bar(x='approach', y='spearmanr', title='Spearman Correlation', figsize=(12, 6)))
display(best_results[['approach', 'kendalltau']].plot.bar(x='approach', y='kendalltau', title='Kendall Correlation', figsize=(12, 6)))